In [1]:
import torch
from torch.utils import data
from torchvision import transforms
import numpy as np
import datetime
import matplotlib.pyplot as plt
from models.stclassifier import dLtae
from models.ltae import LTAE
from learning.focal_loss import FocalLoss
import torchnet as tnt
from learning.metrics import mIou, confusion_matrix_analysis
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from utils import *
# from learn_curve import plot_curve
from altdataset import SITSData
from altdataset import date_positions
from models.stclassifier import dLtae
from models.ltae import LTAE
from learning.focal_loss import FocalLoss
from learning.weight_init import weight_init
from learning.metrics import mIou, confusion_matrix_analysis
import wandb
import os
import json
import glob
import pickle as pkl
import argparse
import pprint
import time
import sys

In [2]:
# date_positions("../../../data/theiaL2A_zip_img/output/2019/gapfilled_dates2019.txt")

## Scenario 3 Workspace

In [3]:
# def date_positions(gfdate_path):
#     with open(gfdate_path, "r") as f:
#         date_list = f.readlines()
#     date_list = [x.strip() for x in date_list]
#     date_list = [datetime.datetime.strptime(x, "%Y%m%d").timetuple().tm_yday for x in date_list]
#     date_ = [x for x in date_list]
#     return date_
# f = date_positions("../../../data/theiaL2A_zip_img/output/2019/gapfilled_dates2019.txt")

In [4]:
# len(f)

In [3]:
m = '../../../results/ltae/model/2018_2019/Seed_0/model.pth.tar'
state_dict = torch.load(m)['state_dict']

In [2]:
# def parameters():
#     parser = argparse.ArgumentParser()

#     # Set-up parameters
#     # parser.add_argument('--dataset_folder', default='../../../data/theiaL2A_zip_img/output/2019', type=str, help='Path to the data folder.') #move npy and date into a folder
#     # parser.add_argument('--res_dir', default='../../../results/ltae/trials', help='Path to the folder where the results should be stored')
#     parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers')
#     parser.add_argument('--seed', default=0, type=int, help='Random seed')
#     parser.add_argument('--device', default='cuda', type=str, help='Name of device to use for tensor computations (cuda/cpu)')
#     parser.add_argument('--display_step', default=100, type=int, help='Interval in batches between display of training metrics')
#     parser.add_argument('--scheduler_', default=False, type=bool, help='Enable scheduler')
#     # parser.add_argument('--preload', dest='preload', action='store_true', help='If specified, the whole dataset is loaded to RAM at initialization')
#     parser.set_defaults(preload=False)
    

#     # Training parameters
#     parser.add_argument('--epochs', default=2, type=int, help='Number of epochs per fold')
#     parser.add_argument('--batch_size', default=128, type=int, help='Batch size')
#     parser.add_argument('--lr', default=0.001, type=float, help='Learning rate')
#     parser.add_argument('--gamma', default=1, type=float, help='Gamma parameter of the focal loss')
#     # parser.add_argument('--weight_decay', default=0, type=float, help='Weight decay rate')

#     ## L-TAE 
#     parser.add_argument('--in_channels', default=10, type=int, help='Number of channels of the input embeddings')
#     parser.add_argument('--n_head', default=16, type=int, help='Number of attention heads')
#     parser.add_argument('--d_k', default=8, type=int, help='Dimension of the key and query vectors')
#     parser.add_argument('--n_neurons', default=[256,128], type=str, help='Number of neurons in the layers of n_neurons')
#     parser.add_argument('--T', default=1000, type=int, help='Maximum period for the positional encoding')
#     parser.add_argument('--positions', default='None', type=str,
#                         help='Positions to use for the positional encoding (bespoke / order)')
#     parser.add_argument('--len_max_seq', default=53, type=int,
#                         help='Maximum sequence length for positional encoding (only necessary if positions == order)')
#     parser.add_argument('--dropout', default=0.2, type=float, help='Dropout probability')
#     parser.add_argument('--d_model', default=256, type=int,
#                         help="size of the embeddings (E), if input vectors are of a different size, a linear layer is used to project them to a d_model-dimensional space")

#     ## Classifier
#     parser.add_argument('--num_classes', default=19, type=int, help='Number of classes')
#     parser.add_argument('--mlp4', default='[128, 64, 32, 19]', type=str, help='Number of neurons in the layers of MLP (Decoder)')
    
#     config = parser.parse_args()
#     config = vars(config)
#     for k, v in config.items():
#         if 'mlp' in k or k == 'nker':
#             v = v.replace('[', '')
#             v = v.replace(']', '')
#             config[k] = list(map(int, v.split(',')))
    
#     return config


In [3]:
# config = parameters()

In [4]:
epochs = 1

In [5]:
res_dir = "../../../results/ltae/model/2018_2019"
seed = 0
def prepare_output():
    os.makedirs(res_dir, exist_ok=True)
    os.makedirs(os.path.join(res_dir, 'Seed_{}'.format(seed)), exist_ok=True)

def checkpoint(log):
    with open(os.path.join(res_dir, 'Seed_{}'.format(seed), 'seed_{}_trainlog.json'.format(seed)), 'w') as outfile:
        json.dump(log, outfile, indent=4)

def save_results(metrics, conf_mat, report):
    with open(os.path.join(res_dir, 'Seed_{}'.format(seed), 'seed_{}_test_metrics.json'.format(seed)), 'w') as outfile:
        json.dump(metrics, outfile, indent=4)
    pkl.dump(conf_mat, open(os.path.join(res_dir, 'Seed_{}'.format(seed), 'seed_{}_conf_mat.pkl'.format(seed)), 'wb'))

    with open(os.path.join(res_dir, 'Seed_{}'.format(seed),'seed_{}_report.txt'.format(seed)), 'w') as f:
        f.write(report)
        f.close()

In [6]:
def train_epoch(model, optimizer, criterion, data_loader, device, config):
    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()
    y_true = []
    y_pred = []

    for i, (x, y) in enumerate(data_loader):
        start_time = time.time()
        y_true.extend(list(map(int, y)))

        x = recursive_todevice(x, device)
        y = y.to(device)
        # print(x.is_cuda)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y.long())
        loss.backward()
        # if config['scheduler_']:
        #     optimizer.step()
        #     scheduler.step()
        # else:
        optimizer.step()
            

        pred = out.detach()
        y_p = pred.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))
        acc_meter.add(pred, y)
        loss_meter.add(loss.item())

        if (i + 1) % config['display_step'] == 0:
            print('Step [{}/{}], Loss: {:.4f}, Acc : {:.2f}'.format(i + 1, len(data_loader), loss_meter.value()[0], acc_meter.value()[0]))
        
        print("Iteration {} completed in {:.4f} second".format(i + 1, time.time() - start_time))
        # if i +1 == int(len(data_loader)/config['factor']):  break
        # if i +1 >= config['factor']:  break
    epoch_metrics = {'train_loss': loss_meter.value()[0],
                     'train_accuracy': acc_meter.value()[0],
                     'train_IoU': mIou(y_true, y_pred, n_classes=19)}
    wandb.log({"train_loss": epoch_metrics['train_loss'], "train_accuracy": epoch_metrics['train_accuracy'], "train_IoU": epoch_metrics['train_IoU']})
    
    return epoch_metrics

def evaluation(model, criterion, loader, device, config, mode='val'):
    y_true = []
    y_pred = []

    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()
    label = ["Dense built-up area", "Diffuse built-up area", "Industrial and commercial areas", "Roads", "Oilseeds (Rapeseed)", "Straw cereals (Wheat, Triticale, Barley)", "Protein crops (Beans / Peas)", "Soy", "Sunflower", "Corn",  "Tubers/roots", "Grasslands", "Orchards and fruit growing", "Vineyards", "Hardwood forest", "Softwood forest", "Natural grasslands and pastures", "Woody moorlands", "Water"]
    for (x, y) in loader:
        start_time = time.time()
        y_true.extend(list(map(int, y)))
        x = recursive_todevice(x, device)
        y = y.to(device)

        with torch.no_grad():
            prediction = model(x)
            loss = criterion(prediction, y)

        acc_meter.add(prediction, y)
        loss_meter.add(loss.item())

        y_p = prediction.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))
        
        print("evaluation iteration completed in {:.4f} seconds".format(time.time() - start_time))
    metrics = {'{}_accuracy'.format(mode): acc_meter.value()[0],
               '{}_loss'.format(mode): loss_meter.value()[0],
               '{}_IoU'.format(mode): mIou(y_true, y_pred, 19)}
    
    if mode == 'val':
        return metrics
    elif mode == 'test':
        return metrics, confusion_matrix(y_true, y_pred, labels=list(range(19))), classification_report(y_true, y_pred, target_names=label, digits=4)

In [7]:
def get_loader(train_dt, val_dt, test_dt):
    
    loader_seq = []
    train_loader = data.DataLoader(train_dt, batch_size=2048,
                                       num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)
    validation_loader = data.DataLoader(val_dt, batch_size=2048,
                                       num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)
    test_loader = data.DataLoader(test_dt, batch_size=2048,
                                       num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)
    loader_seq.append((train_loader, validation_loader, test_loader))
    return loader_seq

def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    else:
        return [recursive_todevice(c, device) for c in x]


In [8]:
data_folder_2018 = "../../../data/theiaL2A_zip_img/output/2018"
mean_ = np.loadtxt(glob.glob(data_folder_2018 + '/*mean.txt')[0])
std_ = np.loadtxt(glob.glob(data_folder_2018 + '/*std.txt')[0])
transform = transforms.Compose([standardize(mean_, std_)])
    
sits_data = glob.glob(os.path.join(data_folder_2018, 'Seed_{}'.format(seed)  + '/*.npz'))
doy = glob.glob(data_folder_2018 + '/gapfilled*.txt')[0]

train_dt = SITSData(sits_data[2], doy, transform = transform)
print("train dataset completed")
val_dt = SITSData(sits_data[1], doy,  transform = transform)
print("val dataset completed")
test_dt = SITSData(sits_data[0], doy,  transform = transform)
print("test dataset completed")
del sits_data

train dataset completed
val dataset completed
test dataset completed


In [9]:
data_folder_2019 = "../../../data/theiaL2A_zip_img/output/2019"
mean_ = np.loadtxt(glob.glob(data_folder_2019 + '/*mean.txt')[0])
std_ = np.loadtxt(glob.glob(data_folder_2019 + '/*std.txt')[0])
transform = transforms.Compose([standardize(mean_, std_)])
    
sits_data_2 = glob.glob(os.path.join(data_folder_2019, 'Seed_{}'.format(seed)  + '/*.npz'))
doy_2 = glob.glob(data_folder_2019 + '/gapfilled*.txt')[0]
    
train_dt_2 = SITSData(sits_data_2[2], doy_2, transform = transform)
print("train dataset completed")
val_dt_2 = SITSData(sits_data_2[1], doy_2,  transform = transform)
print("val dataset completed")
test_dt_2 = SITSData(sits_data_2[0], doy_2,  transform = transform)
print("test dataset completed")
del sits_data_2

train dataset completed
val dataset completed
test dataset completed


In [9]:
train_datasets = []
train_datasets.append((train_dt, train_dt_2))
val_datasets = []
val_datasets.append((val_dt, val_dt_2))
test_datasets = []
test_datasets.append((test_dt, test_dt_2))

In [10]:
trd = data.ConcatDataset(train_datasets)
vd = data.ConcatDataset(val_datasets)
ted = data.ConcatDataset(test_datasets)
loaders_ = get_loader(trd, vd, ted)

In [10]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [11]:
tloader = torch.utils.data.DataLoader(ConcatDataset(train_dt, train_dt_2), batch_size=2048, num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)
# vloader = torch.utils.data.DataLoader(ConcatDataset(val_dt, val_dt_2), batch_size=2048, num_workers=10,
#                                        shuffle=True,
#                                           pin_memory=True)

In [20]:
train_datasets = []
train_datasets.append((train_dt, train_dt_2))

In [21]:
tt = torch.utils.data.ConcatDataset(train_datasets)

In [24]:
tloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dt, train_dt_2]), batch_size=2048, num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)

In [ ]:
torch.utils.data.ConcatDataset

In [25]:
y_true = []
for i, (x,y) in enumerate(tloader):
    y_true.extend(list(map(int,y)))
    break

In [18]:
t = data.DataLoader(ted, batch_size=2048,
                    num_workers=10,
                    shuffle=True,
                    pin_memory=True)

In [24]:
# for _ in tloader:
#     print(_.shape)

In [14]:
for train_loader, val_loader, test_loader in loaders_:
        print('Train {}, Val {}, Test {}'.format(len(train_loader), len(val_loader), len(test_loader)))#, int(len(train_loader)/config['factor'])))

        model_config = dict(in_channels=10, n_head=16, d_k=8,
                            n_neurons=[256,128], dropout=0.2, d_model=256, mlp= [128, 64, 32, 19], T=1000, len_max_seq=53,
                            positions=train_dt.date_positions if 'bespoke' == 'bespoke' else None)
        
        model = dLtae(**model_config)
        wandb.init()
        
        with open(os.path.join(res_dir, 'Seed_{}'.format(seed), 'conf.json'), 'w') as file:
            file.write(json.dumps(config, indent=4))
        # break
        model = model.to(device)
        model.apply(weight_init)
        steps_per_epoch = len(train_loader)
        
        optimizer = torch.optim.Adam(model.parameters())
            
        criterion = FocalLoss(config['gamma'])
        
        model = model.double() #RuntimeError: expected scalar type Double but found Float 
        
        trainlog = {}
        
        best_mIoU = 0
        st_ = time.time()
        for epoch in range(1, epochs + 1):
            print('EPOCH {}/{}'.format(epoch, epochs))
            st__ = time.time()
            model.train()
            
            start_time = time.time()
            # print(torch.get_num_threads())
            # sys.exit()
            train_metrics = train_epoch(model, optimizer, criterion, train_loader, device=device, config=config)
            print("Training time for {} is {}".format(epoch, (time.time() - start_time)/60))
            
            print('Validation . . . ')
            start_time = time.time()
            model.eval()
            val_metrics = evaluation(model, criterion, val_loader, device=device, config=config, mode='val')

            print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(val_metrics['val_loss'], val_metrics['val_accuracy'],
                                                                 val_metrics['val_IoU']))
            print("Validation time for {} is {}".format(epoch, (time.time() - start_time)/60))
            wandb.log({"val_loss": val_metrics['val_loss'], "val_acc": val_metrics['val_accuracy'], "val_IoU": val_metrics['val_IoU']})
            wandb.log({"epoch": epoch})

            trainlog[epoch] = {**train_metrics, **val_metrics}
            checkpoint(trainlog, config)

            if val_metrics['val_IoU'] >= best_mIoU:
                best_mIoU = val_metrics['val_IoU']
                torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()},
                           os.path.join(res_dir, 'Seed_{}'.format(seed), 'model.pth.tar'))
            print("total time taken for {} epoch: {:.3f} mins.".format(epoch, (time.time() - st__)/60))
            
            print('Testing best epoch . . .') #test on the best model only and that should be once.
        model.load_state_dict(
                torch.load(os.path.join(res_dir, 'Seed_{}'.format(seed), 'model.pth.tar'))['state_dict'])
        start_time = time.time()
        model.eval()

        test_metrics, conf_mat, report_ = evaluation(model, criterion, test_loader, device=device, mode='test', config=config)

        print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(test_metrics['test_loss'], test_metrics['test_accuracy'], test_metrics['test_IoU']))
        print("Test time for {} is {}".format(epoch, (time.time() - start_time)/60))
        wandb.log({"test_loss": test_metrics['test_loss'], "test_accuracy": test_metrics['test_accuracy'], "test_IoU": test_metrics['test_IoU']})
        
        
        save_results(test_metrics, conf_mat, report_, config)
        
        print("total time taken for all {} epochs: {:.3f} mins.".format(epochs, (time.time() - st_)/60))

Train 1, Val 1, Test 1


wandb: Currently logged in as: adebowale (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


FileNotFoundError: [Errno 2] No such file or directory: '../../../results/ltae/model/2018_2019/Seed_0/conf.json'

## Test dataset retrival

In [2]:
# from dataset import SITSData

In [5]:
def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    else:
        return [recursive_todevice(c, device) for c in x]

In [6]:
def evaluation(model, criterion, loader, device, config, mode='val'):
    y_true = []
    y_pred = []

    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()

    for (x, y) in loader:
        start_time = time.time()
        y_true.extend(list(map(int, y)))
        x = recursive_todevice(x, device)
        y = y.to(device)

        with torch.no_grad():
            prediction = model(x)
            loss = criterion(prediction, y)

        acc_meter.add(prediction, y)
        loss_meter.add(loss.item())

        y_p = prediction.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))
        
        print("evaluation iteration completed in {:.4f} seconds".format(time.time() - start_time))
    metrics = {'{}_accuracy'.format(mode): acc_meter.value()[0],
               '{}_loss'.format(mode): loss_meter.value()[0],
               '{}_IoU'.format(mode): mIou(y_true, y_pred, config['num_classes'])}
    label = ["Dense built-up area", "Diffuse built-up area", "Industrial and commercial areas", "Roads", "Oilseeds (Rapeseed)", "Straw cereals (Wheat, Triticale, Barley)", "Protein crops (Beans / Peas)", "Soy", "Sunflower", "Corn",  "Tubers/roots", "Grasslands", "Orchards and fruit growing", "Vineyards", "Hardwood forest", "Softwood forest", "Natural grasslands and pastures", "Woody moorlands", "Water"]
    

    if mode == 'val':
        return metrics
    elif mode == 'test':
        return metrics, confusion_matrix(y_true, y_pred, labels=list(range(config['num_classes']))), f1_score(y_true, y_pred, average='macro'), classification_report(y_true, y_pred, target_names=label, digits=4)

In [11]:
# 2018
case="1"
# m = '../../../results/ltae/results/2018/Seed_0/model.pth.tar'
# state_dict = torch.load(m)['state_dict']
c = "../../../results/ltae/results/2018/Seed_0/conf.json"
config = json.load(open(c))


In [12]:
config

{'dataset_folder': '../../../results/ltae',
 'npy': '../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz',
 'dates': 'dates.txt',
 'res_dir': '../../../results/ltae/results/2018',
 'num_workers': 10,
 'seed': 0,
 'device': 'cuda',
 'display_step': 100,
 'factor': 5266,
 'scheduler_': True,
 'epochs': 15,
 'batch_size': 2048,
 'lr': 0.001,
 'gamma': 1,
 'in_channels': 10,
 'n_head': 16,
 'd_k': 8,
 'n_neurons': [256, 128],
 'T': 1000,
 'positions': 'Bespoke',
 'len_max_seq': 33,
 'dropout': 0.2,
 'd_model': 256,
 'num_classes': 19,
 'mlp4': [128, 64, 32, 19],
 'preload': False,
 'N_params': 97319,
 'Train_loader_size': 5266,
 'Val_loader_size': 785,
 'Test_loader_size': 831}

In [6]:
# 2019
case="2"
m = '../../../results/ltae/results/2019/Seed_0/model.pth.tar'
state_dict = torch.load(m)['state_dict']
c = "../../../results/ltae/results/2019/Seed_0/conf.json"
config = json.load(open(c))

In [7]:
# c = "../../../results/ltae/results/2019/Seed_0/conf.json"
# config = json.load(open(c))

In [10]:
# mean_ = np.loadtxt(glob.glob(config['dataset_folder'] + '/*mean.txt')[0])
# std_ = np.loadtxt(glob.glob(config['dataset_folder'] + '/*std.txt')[0])
# transform = transforms.Compose([standardize(mean_, std_)])
    
# sits_data = glob.glob(os.path.join(config['dataset_folder'], 'Seed_{}'.format(config['seed'])  + '/*.npz'))
# doy = glob.glob(config['dataset_folder'] + '/gapfilled*.txt')[0]

In [11]:
# model_ = dLtae(**config)

In [8]:
# call the model
model = dLtae(in_channels = config['in_channels'], n_head = config['n_head'], d_k= config['d_k'], n_neurons=config['n_neurons'], dropout=config['dropout'], d_model= config['d_model'],
                 mlp = config['mlp4'], T =config['T'], len_max_seq = config['len_max_seq'], 
              positions=date_positions if config['positions'] == 'bespoke' else None, return_att=False)

In [9]:
device = "cuda"

In [ ]:
model = model.to(device)
model = model.double()
model.load_state_dict(state_dict)
    
model.eval()

In [11]:
# case 2018
year = "2018"
mean = np.loadtxt('../ltae/mean_std/source_mean.txt')
std = np.loadtxt('../ltae/mean_std/source_std.txt')
npz_ = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"

In [22]:
# 2019
year = "2019"
mean = np.loadtxt('../ltae/mean_std/target_mean.txt')
std = np.loadtxt('../ltae/mean_std/target_std.txt')
npz_ = "../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz" 

In [14]:
npz_ = "../../../data/theiaL2A_zip_img/output/2019" 

In [17]:
print(os.path.basename(npz_).split('.'))

['2019']


In [23]:
# case 2019
# case = "2"
# mean = np.loadtxt('../ltae/mean_std/source_mean.txt')
# std = np.loadtxt('../ltae/mean_std/source_std.txt')
# npz_ = "../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz" 


In [24]:
# case 2018
# case = "3"
# mean = np.loadtxt('../ltae/mean_std/target_mean.txt')
# std = np.loadtxt('../ltae/mean_std/target_std.txt')
# npz_ = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"

In [25]:
transform = transforms.Compose([standardize(mean, std)])

dataset_test_alt = SITSData(npz_, config['seed'], config['dates'], partition='test', transform = transform)

In [26]:
test_loader = data.DataLoader(dataset_test_alt, batch_size=config['batch_size'],
                                       num_workers=config['num_workers'], 
                                        shuffle=True,
                                         pin_memory=True)

In [27]:
# for (x, y) in test_loader:
#     print(x.shape)

In [28]:
print(len(dataset_test_alt))
print(len(test_loader))
print(len(dataset_test_alt)/2048)
print(len(test_loader)*2048)

1518343
742
741.37841796875
1519616


In [29]:
criterion = FocalLoss(config['gamma'])

In [ ]:
test_metrics, conf_mat, fs, report_= evaluation(model, criterion, test_loader, device=device, mode='test', config=config)

In [ ]:
print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(test_metrics['test_loss'], test_metrics['test_accuracy'], test_metrics['test_IoU']))

In [ ]:
with open("reports/LTAE_case_{}_{}_report".format(case, year), "w") as f:
    f.write(report_)
    f.close()

In [45]:
config['res_dir']

'../../../results/ltae/results/2019'

In [23]:
def save_results(metrics, conf_mat, config):
    with open(os.path.join(config['res_dir'], 'Seed_{}'.format(config['seed']), 'seed_{}_case_{}_batchsize_{}_epochs_{}_factor_{}_test_metrics.json'.format(config['seed'], case, config['batch_size'], config['epochs'], config['factor'])), 'w') as outfile:
        json.dump(metrics, outfile, indent=4)
    # pkl.dump(conf_mat, open(os.path.join(config['res_dir'], 'Seed_{}'.format(config['seed']), 'seed_{}_batchsize_{}_epochs_{}_factor_{}_conf_mat.pkl'.format(config['seed'], config['batch_size'], config['epochs'], config['factor'])), 'wb'))

In [24]:
save_results(test_metrics, conf_mat, config)

In [23]:
# m = '../../../results/ltae/trials/Seed_0/model.pth.tar'

# model.load_state_dict(torch.load(m))

In [5]:
def date_positions(gfdate_path):
    with open(gfdate_path, "r") as f:
        out_date_list = f.readlines()
    out_date_list = [x.strip() for x in out_date_list]
    out_date_list = [datetime.datetime.strptime(x, "%Y%m%d").timetuple().tm_yday for x in out_date_list]
    string_date_list = [x for x in out_date_list]
    return string_date_list

In [6]:
# st = time.time()
# t = date_positions("dates.txt")
# s = date_positions("dates.txt")
# print(t)
# print(s)
# print("time: ", time.time() - st)

In [7]:
n_channel = 10

In [10]:
def generate_ids():
    """
    Descr: 
        Aim: To write and returns the partition (Train, Validation and Test) ids 
        with respect to the grid split index (range 1 - 100)
        
        - A random seed value is set within a random intger 1-10,
        - the set is spltted into 80:10:10,
        - save into a text file with the seed value used
    """
    # set a random seed value within the range 1 -10 
    start_time = time.time()
    seed_value = 4
    # seed_value = np.random.randint(0,10)
    np.random.seed(seed_value)
    # # block id range 1 - 100 (splitted grid)
    block_range = np.arange(1, 101)

    # Train, Validation and Test
    random.shuffle(block_range)
    train_id = block_range[:60] # 60%
    val_id = block_range[60:70] # 10%
    test_id = block_range[70:] # 30%
    print("Seed value: ", seed_value)
    
    if not os.path.exists("./ids/train_val_eval_seed_" + str(seed_value)+".txt"):
        with open("./ids/train_val_eval_seed_" + str(seed_value)+".txt", "w") as f:
            f.write("Training: " + str(list(train_id)) + "\n")
            f.write("Validation: " + str(list(val_id)) + "\n")
            f.write("Testing: " + str(list(test_id)) + "\n")
            f.close()
    print('Read set ids completed: %s second' % (time.time() - start_time))
generate_ids()

Seed value:  4
Read set ids completed: 0.004056215286254883 second


In [3]:
def read_ids(seed_value):
    """
    Read ids from file
    """
    assert seed_value >= 0 and seed_value <= 10
    
    with open("./ids/train_val_eval_seed_" + str(seed_value)+".txt", "r") as f:
        lines = f.readlines()
        Train_ids = eval(lines[0].split(":")[1])
        Val_ids = eval(lines[1].split(":")[1])
        test_ids = eval(lines[2].split(":")[1])
    return Train_ids, Val_ids, test_ids

In [4]:
Train_ids, Val_ids, test_ids = read_ids(0)

In [9]:
j = Val_ids + test_ids

In [2]:
import glob
s = glob.glob("../../../data/theiaL2A_zip_img/output/2018/Seed_0" + "/*.npz")

In [5]:
s[1]

'../../../data/theiaL2A_zip_img/output/2018/Seed_0/val.npz'

In [2]:
def compute_mean_std(source_sits, target_sits, case):
    """
    Descr: Compute mean and std for each channel
    Input: both SITS dataset(.npz) paths
            Case[1 - 3]:
            1 - concatenate both dataset, while 2 & 3 rep source and target respectively
    The data(from N,LxD) is reshaped into (N,D,L);
        where N - pixel, D - Bands (10), L - Time (33)
    
    """
    
    # case = 1: both, case = 2: target, case = 3: target
    if case == 1:
        sits = [source_sits, target_sits]
    elif case == 2:
        sits = source_sits
    elif case == 3:
        sits = target_sits
    else:
        print('Select case between 1-3')
        return None
    
    # if sits is a list, then it's a list of paths
    if isinstance(sits, list):
        # load data
        X_source = np.load(sits[0])['X']
        X_target = np.load(sits[1])['X']
        # concatenate the data
        X = np.concatenate((X_source, X_target), axis=0)
    # if sits is a string, then it's a path
    else: 
        with np.load(sits) as data:
            X = data['X']

    X = X.reshape(X.shape[0], n_channel, int(X.shape[1]/n_channel))
    # compute mean and std
    X_mean = np.mean(X, axis=(0,2))
    X_std = np.std(X, axis=(0,2))
    print('mean shape: ', X_mean.shape)
    print('std shape: ', X_std.shape)
    # save X_mean and X_std sepearately for sits as txt file
    np.savetxt(os.path.join('mean_'+ str(case) +'.txt'), X_mean)
    np.savetxt(os.path.join('std_'+ str(case) +'.txt'), X_std)

# for i in [1,2,3]:
#     start_time = time.time()
#     source_path = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
#     target_path = "../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz"
#     compute_mean_std(source_path, target_path, i)
#     print("run time: ", time.time() - start_time)

In [3]:
import os
import numpy as np
import time
def compute_mean_stdv2(sits, domain='source'):
    """
    Descr: Compute mean and std for each channel
    Input: both SITS dataset(.npz) paths
    The data(from N,LxD) is reshaped into (N,D,L);
        where N - pixel, D - Bands (10), L - Time (33)
    """
    start_time = time.time()
    with np.load(sits) as data:
            X = data['X']
    print("data completed: ", time.time() - start_time)
    
    n_channel = 10
    
    X = X.reshape(X.shape[0], n_channel, int(X.shape[1]/n_channel))
    # compute mean and std
    X_mean = np.mean(X, axis=(0,2))
    X_std = np.std(X, axis=(0,2))
    print('mean shape: ', X_mean.shape)
    print('std shape: ', X_std.shape)
    # save X_mean and X_std sepearately for sits as txt file
    np.savetxt(os.path.join('./mean_std/', domain + '_mean.txt'), X_mean)
    np.savetxt(os.path.join('./mean_std/', domain + '_std.txt'), X_std)

source_path = '../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz'
# target_path = '../../../data/theiaL2A_zip_img/output/2019/2019_SITS_subset_data.npz'
compute_mean_stdv2(source_path, '2019')
# compute_mean_stdv2(target_path, domain = 'target')

data completed:  92.74277639389038
mean shape:  (10,)
std shape:  (10,)


In [13]:
# class SITSDatav2(data.Dataset):
#     def __init__(self, sits, seed, partition='train', transform=None):
        
#         self.sits = sits
#         self.seed = seed
#         self.transform = transform
        
#         # get partition ids using the read_id() func
#         start_time = time.time()
#         self.train_ids, self.val_ids, self.test_ids = read_ids(self.seed)
#         print("read ids completed: %s second" % (time.time() - start_time))

#         # select partition
#         if partition == 'train':
#             ids = self.train_ids
#         elif partition == 'val':
#             ids = self.val_ids
#         elif partition == 'test':
#             ids = self.test_ids
#         else:
#             raise ValueError('Invalid partition: {}'.format(partition))
        
#         start_time = time.time()
#         print('reading files....')
#         X, y, block_ids = load_npz(self.sits)
#         print("load npz: %s seconds" % (time.time() - start_time))
        
#         start_time = time.time()
#         y = np.unique(y, return_inverse=True)[1]
#         print("reassigning %s seconds" % (time.time() - start_time))
        
#         # concatenate the data
#         start_time = time.time()
#         data_ = np.concatenate((X, y[:, None], block_ids[:, None]), axis=1)
#         print("Concatenating completed: %s seconds" % (time.time() - start_time))

#         # filter by block_id
#         start_time = time.time()
#         data_ = data_[np.isin(data_[:, -1], ids)]
#         print("filtering ids completed: %s seconds" % (time.time() - start_time))
        
#         self.X_ = data_[:, :-2]
#         self.y_ = data_[:, -2]                          
        
#         del X
#         del y
#         del block_ids
#         del data_

#     def __len__(self):
#         return len(self.y_)

#     def __getitem__(self, idx):
#         start_time = time.time()
#         self.X = self.X_[idx]
#         self.y = self.y_[idx]
#         print("getting data: %s seconds" % ((time.time() - start_time)*100))

#         start_time = time.time()
#         self.X = np.array(self.X).astype('float32')
#         self.y = np.array(self.y).astype('float32')
#         print("conversion: %s seconds" % ((time.time() - start_time)*100))
        
#         start_time = time.time()
#         self.X = self.X.reshape(int(self.X.shape[0]/n_channel), n_channel)
#         print("reshape data: %s seconds" % ((time.time() - start_time)*100))

#         # transform
#         start_time = time.time()
#         if self.transform:
#             self.X = self.transform(self.X)
#         print("transform data: %s seconds" % ((time.time() - start_time)*100))
#         print(self.X.shape)
        
#         start_time = time.time()
#         torch_x = torch.from_numpy(self.X)
#         torch_y = torch.from_numpy(self.y)
#         print("tensor: %s seconds" % ((time.time() - start_time)*100))
        
#         return torch_x, torch_y

In [3]:
with np.load("../../../data/theiaL2A_zip_img/output/2018/Seed_0/test.npz") as f:
    X = f['X']
    y = f['y']

In [11]:
train_ids, val_ids, test_ids = read_ids(0)

In [ ]:
y = np.unique(y, return_inverse=True)[1]

In [1]:
from utils import *
import numpy as np
import os
def separate_data(sits, seed, outdir):

    train_ids, val_ids, test_ids = read_ids(seed)

    # if partition == 'train':
    #     ids = train_ids
    # elif partition == 'val':
    #     ids = val_ids
    # elif partition == 'test':
    #     ids = test_ids
    # else:
    #     raise ValueError('Invalid partition: {}'.format(partition))

    X, y, block_ids = load_npz(sits)
    print("npz....")

    y = np.unique(y, return_inverse=True)[1] # reassigning label [1,23] to [0,18]
    print("reaass....")

    # concatenate the data
    data_ = np.concatenate((X, y[:, None], block_ids[:, None]), axis=1)
    print("concatenating....")
    
    del X
    del y

    def pickk(partition, dt_):
        
        if partition == 'train':
            ids = train_ids
        elif partition == 'val':
            ids = val_ids
        elif partition == 'test':
            ids = test_ids
        else:
            raise ValueError('Invalid partition: {}'.format(partition))
        
        print("filter....")    
        dat = dt_[np.isin(dt_[:, -1], ids)]
        print("filter done....")    
        
        print("selecting....")    
        X_ = dat[:, :-2]
        y_ = dat[:, -2]
        print("selecting done....")    
        
        print("savinng....")    
        np.savez_compressed(os.path.join(outdir, "{}.npz".format(partition)), X=X_, y=y_)
        print("completed....")    
    pickk('train', data_)
    print("1 done....")    
    pickk('val', data_)
    print("2 done....")    
    pickk('test', data_)
    print("3 done....")    

    # train_data = data_[np.isin(data_[:, -1], train_ids)]
    # np.save(os.path.join(outdir, "Xtrain.npy".format(partition)), X_)
    # np.save(os.path.join(outdir, "ytrain.npy")), y_)
    # val_data = data_[np.isin(data_[:, -1], val_ids)]
    # test_data = data_[np.isin(data_[:, -1], test_ids)]

#     X_ = data_[:, :-2]
#     y_ = data_[:, -2]
    
#     np.save(os.path.join(outdir, "X{}.npy".format(partition)), X_)
#     np.save(os.path.join(outdir, "y{}.npy".format(partition)), y_)

# def prepare_output():
#     os.makedirs(os.path.join(outdir,  exist_ok=True)
#     os.makedirs(os.path.join(config['res_dir'], 'Seed_{}'.format(config['seed'])), exist_ok=True)

In [2]:
sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
seed =0
# partition = "train"
outdir = "../../../data/theiaL2A_zip_img/output/2018/Seed_0"
separate_data(sits, seed, outdir)

npz....
reaass....
concatenating....
filter....
filter done....
selecting....
selecting done....
savinng....
completed....
1 done....
filter....
filter done....
selecting....
selecting done....
savinng....
completed....
2 done....
filter....
filter done....
selecting....
selecting done....
savinng....
completed....
3 done....


In [ ]:
# sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
# seed =0
# partition = "val"
# outdir = "../../../data/theiaL2A_zip_img/output/2018/Seed_0"
# separate_data(sits, seed, partition, outdir )

In [ ]:
# sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
# seed =0
# partition = "test"
# outdir = "../../../data/theiaL2A_zip_img/output/2018/Seed_0"
# separate_data(sits, seed, partition, outdir )

In [8]:
class standardize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        return (sample - self.mean) / self.std

In [9]:
# np.loadtxt(glob.glob("../../../data/theiaL2A_zip_img/output/2018" + "/gapfilled*.txt")[0])

In [10]:
# def __init__(self, sits, seed, date_, partition='train', transform=None):

In [12]:
import glob
mean = np.loadtxt('./mean_std/2018_mean.txt')
std = np.loadtxt('./mean_std/2018_std.txt')
seed = 0 
transform = transforms.Compose([standardize(mean, std)])

# paths
source_path = '../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz'
# target_path = '../../../data/theiaL2A_zip_img/output/2019/2019_SITS_subset_data.npz'

# doy
doy = glob.glob("../../../data/theiaL2A_zip_img/output/2018" + "/gapfilled*.txt")[0]
train_dataset = SITSData(source_path, seed, doy, partition='train', transform=transform)
# val_dataset = SITSDatav2(source_path, seed, partition='val', transform=transform)
# test_dataset = SITSDatav2(source_path, seed, partition='test', transform=transform)

init completed...


In [14]:
train_datasetv = SITSData(source_path, seed, doy, partition='train', transform=transform)

init completed...


In [15]:
test_dataset = SITSData(source_path, seed, doy, partition='test', transform=transform)

init completed...


In [13]:
train_loader = data.DataLoader(train_dataset, batch_size=2048,
                                       num_workers=10,
                                       shuffle=True,
                                          pin_memory=True)

In [3]:
start_time = time.time()
source_path = '../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz'
X, y, _ = load_npz(source_path)
print(time.time() - start_time)

93.67607998847961
